In [1]:
import json
import time
from typing import Dict, Any, Optional
from enum import Enum

class HTTPError(Exception):
    """HTTP 관련 예외"""
    def __init__(self, status_code, message, response_data=None):
        super().__init__(message)
        self.status_code = status_code
        self.response_data = response_data

class NetworkError(Exception):
    """네트워크 관련 예외"""
    pass

class AuthenticationError(HTTPError):
    """인증 오류"""
    def __init__(self, message="인증에 실패했습니다"):
        super().__init__(401, message)

class RateLimitError(HTTPError):
    """요청 제한 오류"""
    def __init__(self, retry_after=None):
        message = f"요청 한도를 초과했습니다. {retry_after}초 후 재시도하세요." if retry_after else "요청 한도를 초과했습니다."
        super().__init__(429, message)
        self.retry_after = retry_after

class APIClient:
    """웹 API 클라이언트 (시뮬레이션)"""
    
    def __init__(self, base_url, api_key, max_retries=3):
        self.base_url = base_url
        self.api_key = api_key
        self.max_retries = max_retries
        self.session_data = {}
    
    def _simulate_http_request(self, endpoint, method="GET", data=None):
        """HTTP 요청 시뮬레이션"""
        import random
        
        # 다양한 응답 시뮬레이션
        random_value = random.random()
        
        if random_value < 0.1:  # 10% 확률로 네트워크 오류
            raise NetworkError("네트워크 연결에 실패했습니다")
        elif random_value < 0.15:  # 5% 확률로 인증 오류
            raise AuthenticationError("API 키가 유효하지 않습니다")
        elif random_value < 0.2:  # 5% 확률로 rate limit
            raise RateLimitError(60)  # 60초 후 재시도
        elif random_value < 0.25:  # 5% 확률로 서버 오류
            raise HTTPError(500, "내부 서버 오류")
        elif random_value < 0.3:  # 5% 확률로 Not Found
            raise HTTPError(404, f"엔드포인트를 찾을 수 없습니다: {endpoint}")
        else:
            # 성공 응답
            return {
                "status": "success",
                "data": f"Response from {endpoint}",
                "timestamp": time.time()
            }
    
    def make_request(self, endpoint, method="GET", data=None, retry_count=0):
        """재시도 로직이 포함된 요청"""
        
        try:
            logger.info(f"API 요청: {method} {endpoint} (시도 {retry_count + 1})")
            
            response = self._simulate_http_request(endpoint, method, data)
            logger.info(f"API 요청 성공: {endpoint}")
            return response
            
        except NetworkError as e:
            logger.warning(f"네트워크 오류: {e}")
            
            if retry_count < self.max_retries:
                wait_time = 2 ** retry_count  # 지수 백오프
                logger.info(f"{wait_time}초 후 재시도합니다...")
                time.sleep(wait_time)
                return self.make_request(endpoint, method, data, retry_count + 1)
            else:
                logger.error(f"최대 재시도 횟수 초과: {self.max_retries}")
                raise
        
        except RateLimitError as e:
            logger.warning(f"요청 제한: {e}")
            
            if retry_count < self.max_retries and e.retry_after:
                logger.info(f"{e.retry_after}초 후 재시도합니다...")
                time.sleep(min(e.retry_after, 60))  # 최대 60초만 기다림
                return self.make_request(endpoint, method, data, retry_count + 1)
            else:
                raise
        
        except HTTPError as e:
            if e.status_code >= 500:  # 서버 오류는 재시도
                logger.warning(f"서버 오류 ({e.status_code}): {e}")
                
                if retry_count < self.max_retries:
                    wait_time = 2 ** retry_count
                    logger.info(f"{wait_time}초 후 재시도합니다...")
                    time.sleep(wait_time)
                    return self.make_request(endpoint, method, data, retry_count + 1)
                else:
                    logger.error(f"최대 재시도 횟수 초과: {self.max_retries}")
                    raise
            else:
                # 클라이언트 오류는 재시도하지 않음
                logger.error(f"클라이언트 오류 ({e.status_code}): {e}")
                raise
        
        except Exception as e:
            logger.error(f"예상치 못한 오류: {type(e).__name__}: {e}")
            raise

class UserService:
    """사용자 서비스 - API 클라이언트 활용"""
    
    def __init__(self, api_client: APIClient):
        self.api_client = api_client
    
    def get_user_profile(self, user_id: str) -> Optional[Dict[str, Any]]:
        """사용자 프로필 조회"""
        try:
            response = self.api_client.make_request(f"/users/{user_id}")
            return response.get("data")
            
        except AuthenticationError as e:
            logger.error(f"인증 실패로 사용자 프로필 조회 불가: {e}")
            return None
            
        except HTTPError as e:
            if e.status_code == 404:
                logger.warning(f"사용자를 찾을 수 없습니다: {user_id}")
                return None
            else:
                logger.error(f"사용자 프로필 조회 실패: {e}")
                raise
        
        except Exception as e:
            logger.error(f"사용자 프로필 조회 중 예상치 못한 오류: {e}")
            raise
    
    def create_user(self, user_data: Dict[str, Any]) -> bool:
        """사용자 생성"""
        try:
            response = self.api_client.make_request("/users", "POST", user_data)
            logger.info(f"사용자 생성 성공: {user_data.get('username')}")
            return True
            
        except AuthenticationError as e:
            logger.error(f"인증 실패로 사용자 생성 불가: {e}")
            return False
            
        except HTTPError as e:
            if e.status_code == 400:
                logger.warning(f"잘못된 사용자 데이터: {user_data}")
                return False
            elif e.status_code == 409:
                logger.warning(f"이미 존재하는 사용자: {user_data.get('username')}")
                return False
            else:
                logger.error(f"사용자 생성 실패: {e}")
                raise
        
        except Exception as e:
            logger.error(f"사용자 생성 중 예상치 못한 오류: {e}")
            raise

def test_api_client():
    """API 클라이언트 테스트"""
    
    # API 클라이언트 생성
    client = APIClient("https://api.example.com", "test-api-key", max_retries=2)
    user_service = UserService(client)
    
    # 테스트 케이스들
    test_cases = [
        ("사용자 조회", lambda: user_service.get_user_profile("user123")),
        ("사용자 생성", lambda: user_service.create_user({"username": "newuser", "email": "test@example.com"})),
        ("다른 사용자 조회", lambda: user_service.get_user_profile("user456")),
        ("또 다른 사용자 생성", lambda: user_service.create_user({"username": "anotheruser", "email": "another@example.com"}))
    ]
    
    print("=== API 클라이언트 테스트 ===")
    
    for test_name, operation in test_cases:
        print(f"\n--- {test_name} ---")
        try:
            result = operation()
            print(f"✅ 성공: {result}")
        except Exception as e:
            print(f"❌ 실패: {type(e).__name__}: {e}")

test_api_client()


=== API 클라이언트 테스트 ===

--- 사용자 조회 ---
❌ 실패: NameError: name 'logger' is not defined

--- 사용자 생성 ---
❌ 실패: NameError: name 'logger' is not defined

--- 다른 사용자 조회 ---
❌ 실패: NameError: name 'logger' is not defined

--- 또 다른 사용자 생성 ---
❌ 실패: NameError: name 'logger' is not defined
